In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop



ss = SparkSession.builder \
    .appName("sta321") \
    .config("master", "local[4]") \
    .enableHiveSupport() \
    .getOrCreate()
sc = ss.sparkContext
ss.conf.set('spark.sql.repl.eagerEval.enabled', True)


22/11/15 14:45:08 WARN Utils: Your hostname, XunyiJiangdeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.12.7.110 instead (on interface en0)
22/11/15 14:45:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/15 14:45:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
zones = ss.read.csv('taxi_zones.csv',header=True)
data = ss.read.csv('taxi-data.csv',header=True)


In [4]:
zones.count()
data.select("PULocationID","DOLocationID").distinct().count()
#34980

32729

# 数据概览

每条数据包括以下 18 个特征:
– VendorID: 数据来源的公司，1=Creative Mobile Technologies;
2=VeriFone Inc.

– tpep_pickup_datetime: 开始计价的时间

– tpep_dropoff_datetime: 关闭计价的时间

– passenger_count: 乘客数量

– trip_distance: 路程(英里)

– RatecodeID: 最终费率 code


1. 标准价格
2. JFK
3. Newark
4. Nassau or Westchester
5. Negotiated fare
6. Group ride
– store_and_fwd_flag: 行程记录是否在发送给供应商之前存储在车辆中

• Y=存储和转发
• N=非存储和转发
– PULocationID: 起点 ID(连表查询，下同)

– DOLocationID:终点 ID

– payment_type: 付款方式
1. 信用卡
2. 现金
3. 免费
4. 有争议 5. 未知
6. 无效行程

– fare_amount: 费用($)

– extra:杂项额外费用和附加费

– mta_tax: 根据使用计费率自动触发的$0.5 的附加费

– tip_amount: 小费

– tolls_amount: 所有通行费用总金额

– improvement_surcharge: $0.3 的改进附加费用

– total_amount: 总费用

– congestion_surcharge: 拥堵费


地点 ID 查询表:
– 该表有四列

– LocationID: 地点 ID

– Borough:自治市/镇

– Zone: 区

– service_zone: 服务区

# 课题一
请计算各 LocationID 之间的路程，并分析各时间段路径繁忙/拥堵程度，给出相应的可视化展示。

1. 计算location ID之间的距离
- 看一共有多少组
- 看对应每组地点之间的时间
   - 不分顺序求均值，最大，最小，quantiles
   - 分顺序求以上的统计量

2. 分析各时间段的繁忙/拥堵程度




### 计算location之间的距离
- dis1 用于得到起点终点不同的距离
    - 对平均值画出直方图：想看大值的里程数分布是怎么样的

- dis2 表示同一段路程中的距离

In [5]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
data = data.withColumn('trip_distance',data.trip_distance.cast(IntegerType()))

In [254]:
data.count() # 6896317
data.drop_duplicates().count() # 6896317, 说明这个每行都一不一样, 6896317yes!

6896317

In [21]:
dis1 = data.groupBy('PULocationID','DOLocationID') \
    .agg(sum("trip_distance").alias("sum_d"), \
         avg("trip_distance").alias("avg_d"), \
         min("trip_distance").alias("min_d"), \
         max("trip_distance").alias("max_d"), \
         stddev("trip_distance").alias("std_d"),\
         count("VendorID").alias("num")
     )


- 从groupby后的表格中left join
    - 含有sum+num+avg
- 整理每一行的名字， 得到dis2: 这个表中存储着A-B的距离，其中1表示该方向，2表示反方向，null则代表没有这个方向的数据
- 处理null值，d2,num2 可以直接设置为0, min2 设置为一个最大数即可, max2 设置成最小值
- 通过将sum1 + sum2再除以总和得到均值


TODO：如何直接在原始数据上得到两个距离之间的路程？并且通过这个计算出这个距离的基本统计量？

In [59]:
dis2 = dis1.alias("data1").join(dis1.alias("data2"), \
    (col("data1.PULocationID") == col("data2.DOLocationID")) & (col("data2.PULocationID") == col("data1.DOLocationID")), 'left') \
    .select(col("data1.PULocationID").alias("PULocationID"),\
        col("data1.DOLocationID").alias("DOLocationID"),\
        col("data1.sum_d").alias("d1"),\
        col("data2.sum_d").alias("d2"),\
        col("data1.num").alias("num1"),\
        col("data2.num").alias("num2"),\
        col("data1.min_d").alias("min1"),\
        col("data2.min_d").alias("min2"),\
        col("data1.max_d").alias("max1"),\
        col("data2.max_d").alias("max2")
        )   
#    .count() -- 32729 如果是left join, 然后再
#  dis2.filter(dis2.num2.isNull()).count() 有7857行是null，所以将 32729+7857 =40586/2 =>20293 对 ?20417


In [80]:
# 填充空值
# 对于空的处理 null
# - d2,num2 可以直接设置为0
# min2 设置为一个最大数即可
# dis3 是更改了null的
import sys

dis3 = dis2.na.fill(value=0, subset= ["d2","num2","min2","max2"])\
    .na.fill(value = sys.maxsize, subset=["min2"])\
    .na.fill(value = - sys.maxsize, subset=["max2"])


In [81]:
dis2_avg = dis3.selectExpr("PULocationID","DOLocationID",\
    "(d1+d2)/(num1+num2) as avg","min1","min2", "max1","max2")

# 检查是否正确
# dis2_avg.filter((dis2_avg["PULocationID"] == '148')& (dis2_avg["DOLocationID"] == "141") | (dis2_avg["PULocationID"] == '141')& (dis2_avg["DOLocationID"] == "148"))


In [82]:
dis3

PULocationID,DOLocationID,d1,d2,num1,num2,min1,min2,max1,max2
148,141,3520,1599,909,376,1,0,13,9
230,262,3426,1768,1287,649,0,0,11,7
234,13,5717,4348,1954,1384,0,-3,13,11
211,88,411,425,253,254,1,1,4,5
158,246,2727,2727,2946,2709,0,0,5,6
231,264,132,890,169,284,0,0,14,29
231,4,1345,447,585,219,-3,-3,7,6
100,260,724,91,127,19,4,0,17,6
246,230,7398,8893,6502,8209,0,0,24,21
170,41,1813,442,393,94,0,0,11,7


In [117]:
dis3.count()

32729

In [118]:
# 将 dataframe 变成rdd, 然后变成一个sorted list， 如何保留方向信息？ 我觉得直接加上一行，which is sorted
# dis4 是drop-duplicated之后的
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
def get_end2end(values):
    return Row(values[0], values[1], values[2],values[3], values[4],values[5],values[6],values[7],values[8],values[9], ','.join(sorted([values[0], values[1]])))

rdd1 = dis3.rdd.map(get_end2end)
dis4 = rdd1.toDF(['PULocationID','DOLocationID','d1','d2','num1','num2','min1','min2','max1','max2','end2end'])\
    .drop_duplicates(subset=['end2end'])
dis4_avg = dis4.selectExpr("PULocationID","DOLocationID",\
    "(d1+d2)/(num1+num2) as avg","min1","min2", "max1","max2","end2end")
dis4_avg

PULocationID,DOLocationID,avg,min1,min2,max1,max2,end2end
1,1,0.8951507208387942,0,0,26,26,"1,1"
10,1,33.666666666666664,28,0,39,0,"1,10"
100,1,16.794486215538846,0,0,21,0,"1,100"
107,1,15.99705882352941,7,0,24,0,"1,107"
11,1,18.0,18,0,18,0,"1,11"
112,1,18.0,18,0,18,0,"1,112"
113,1,14.451612903225806,12,14,20,14,"1,113"
114,1,13.94017094017094,12,0,22,0,"1,114"
116,1,23.4,21,0,26,0,"1,116"
119,1,24.0,24,0,24,0,"1,119"


In [115]:
dis4_avg.count()
# 不知道为什么对不上。。。

20417

#### 在最开始就用rdd 转化一下

In [126]:
def get_end2end2(values):
    return Row(values[0], values[1], values[2], ','.join(sorted([values[0], values[1]])))

dis5 = data.select('PULocationID','DOLocationID','trip_distance').rdd.map(get_end2end2)\
    .toDF(['PULocationID','DOLocationID','trip_distance','end2end'])\
    .drop_duplicates(subset = ['end2end'])\
    .groupBy('end2end')\
    .agg(sum("trip_distance").alias("sum_d"), \
         avg("trip_distance").alias("avg_d"), \
         min("trip_distance").alias("min_d"), \
         max("trip_distance").alias("max_d"), \
         stddev("trip_distance").alias("std_d"),\
         count("end2end").alias("num")
     )
     

In [128]:
dis5

end2end,sum_d,avg_d,min_d,max_d,std_d,num
"113,142",3,3.0,3,3,null,1
"142,209",7,7.0,7,7,null,1
"163,170",1,1.0,1,1,null,1
"161,233",0,0.0,0,0,null,1
"107,97",4,4.0,4,4,null,1
"17,239",10,10.0,10,10,null,1
"231,75",7,7.0,7,7,null,1
"146,249",7,7.0,7,7,null,1
"147,263",4,4.0,4,4,null,1
"145,264",0,0.0,0,0,null,1


### 分析拥堵程度
分析各时间段路径繁忙/拥堵程度
- 首先把时间按照区间分：早高峰（7:00-10:00），之间时间，白天（10:00-17:00），晚高峰（17:00-20:00），晚上（20:00-7:00）
   - 按空格截取时间
   - 按照日期附上星期
   - 通过工作日和节假日进行区分
   - 分别对工作日和节假日统计上述时段的打车次数

- 定义繁忙程度：
   - 总和-可以得到
   - 根据每个路段的平均速度：距离/时间差



In [131]:
data.describe()
#select(min("tpep_pickup_datetime"),max("tpep_pickup_datetime"))


summary,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6845299,6896317,6896317,6845299,6896317,6845299,6845299,6896317,6896317,6845299,6896317,6896317,6896317,6896317,6896317,6896317,6896317,6896317
mean,1.6664569363588062,null,null,1.5508773247158378,2.488633715648512,1.0657559297263712,null,163.65246957180187,161.4892224356856,1.2989929293081282,13.590267515254224,1.108518423674553,0.4924147425357621,2.2446201080963277,0.3881238289943238,0.297985707438931,19.641648841108157,2.275277731577594
stddev,0.47147865361194635,null,null,1.174329936324991,16.433577726066975,0.9309869500238785,null,66.057584252846,70.33752677935428,0.48794008332885574,152.26922211518502,1.259892096450168,0.07233856758938441,2.875127115256801,1.7155019763696664,0.03381443693976937,152.52358091981944,0.7359668903723857
min,1,2008-12-31 18:13:42,2008-12-31 18:27:42,0,-37264,1,N,1,1,1,-0.01,-0.5,-0.5,-0.01,-1,-0.3,-0.31,-0.75
max,2,2090-12-31 06:41:26,2090-12-31 07:18:49,9,19130,99,Y,99,99,5,99.99,90.06,3.3,99.99,97,0.3,99.99,3


In [239]:
import re
import datetime

# 2008-12-31 18:13:42
a = [int(i) for i in re.split("\W+", "2008-12-31 18:13:42")]
datetime.date(a[0], a[1], a[2]).isoweekday()
b = [int(i) for i in re.split("\W+", "2009-01-02 18:13:42")]
timedelta = datetime.datetime(a[0],a[1],a[2],a[3],a[4],a[5]) - datetime.datetime(b[0],b[1],b[2],b[3],b[4],b[5])
timedelta.seconds



0

In [247]:
def get_weekday(values):
    """这个方法用来将日期对应到weekday，以及计算出时间差
    1. 如果发现end-start==0， 那么将这个时间差复值为-1
    2. 如果end-start 刚好是一整天，则会存在问题，需要把days转化为小时，mph，全称miles per hour
 
    Args:
        values (_type_): _description_

    Returns:
        _type_: _description_
    """
    temp1 = [int(i) for i in re.split("\W+",values[3])]
    temp11 = values[3].split(' ')
    start = datetime.datetime(temp1[0],temp1[1],temp1[2],temp1[3],temp1[4],temp1[5])
    weekday1 = datetime.date(temp1[0], temp1[1], temp1[2]).isoweekday()


    temp2 = [int(i) for i in re.split("\W+",values[4])]
    temp22 = values[4].split(' ')
    end = datetime.datetime(temp2[0],temp2[1],temp2[2],temp2[3],temp2[4],temp2[5])
    weekday2 = datetime.date(temp2[0], temp2[1], temp2[2]).isoweekday()

    delta_time = 0 # 单位是小时h
    if ((end-start).seconds  == 0) & ((end-start).days == 0):
        delta_time = -1
    elif ((end-start).seconds  == 0) & ((end-start).days != 0):
        delta_time = (end-start).days * 24
    else:
        delta_time = (end-start).seconds/3600

    # 计算速度
    speed = int(values[2]/delta_time) # mph

    

    # 返回'PULocationID','DOLocationID','trip_distance','pickup_date','pickup_time','pickup_weekday','dropoff_date','dropoff_time','dropoff_weekday'
    return Row(values[0], values[1], values[2], temp11[0], temp11[1], weekday1,temp22[0], temp22[1], weekday2,delta_time,speed)


    
weekday_df = data.select('PULocationID','DOLocationID','trip_distance', \
    'tpep_pickup_datetime',	'tpep_dropoff_datetime').rdd.map(get_weekday)\
    .toDF(['PULocationID','DOLocationID','trip_distance','pickup_date','pickup_time'\
        ,'pickup_weekday','dropoff_date','dropoff_time','dropoff_weekday',"delta_time","speed_mph"])


#### 看是否有非法数据

In [179]:
# weekday_df.filter(weekday_df['pickup_week'] != weekday_df['dropoff_weekday']).count() --77023
# weekday_df.filter(weekday_df['pickup_date'] != weekday_df['dropoff_date']).count() --77023 跨天的
# 看有没有非法数据，比如间隔很多天：0
# weekday_df.filter((weekday_df['dropoff_date'] - weekday_df['pickup_date'] != 1) | (weekday_df['dropoff_date'] - weekday_df['pickup_date'] != -6)).count()


0

#### 对时间段进行处理以及得到平均速度
- 首先将时间映射到4个时间上（用rdd, map 实现）
- 然后通过count每一组的数量得到整个的(groupby time_interval)
- 进一步看比较繁忙的路段的（groupby time_interval and 起点终点）

In [248]:
weekday_df

PULocationID,DOLocationID,trip_distance,pickup_date,pickup_time,pickup_weekday,dropoff_date,dropoff_time,dropoff_weekday,delta_time,speed_mph
142,116,4,2019-12-01,00:26:58,7,2019-12-01,00:41:45,7,0.24638888888888888,16
145,145,0,2019-12-01,00:12:08,7,2019-12-01,00:12:14,7,0.001666666666666...,0
145,145,0,2019-12-01,00:25:53,7,2019-12-01,00:26:04,7,0.003055555555555...,0
138,25,9,2019-12-01,00:12:03,7,2019-12-01,00:33:19,7,0.35444444444444445,25
161,237,1,2019-12-01,00:05:27,7,2019-12-01,00:16:32,7,0.18472222222222223,5
161,230,1,2019-12-01,00:58:51,7,2019-12-01,01:08:37,7,0.16277777777777777,6
164,163,1,2019-12-01,00:14:19,7,2019-12-01,00:27:06,7,0.21305555555555555,4
79,224,0,2019-12-01,00:29:35,7,2019-12-01,00:32:29,7,0.04833333333333333,0
79,107,1,2019-12-01,00:42:19,7,2019-12-01,00:50:34,7,0.1375,7
148,4,0,2019-12-01,00:19:48,7,2019-12-01,00:24:18,7,0.075,0


In [250]:
#PULocationID	DOLocationID	trip_distance	pickup_date	pickup_time	pickup_weekday	dropoff_date	dropoff_time	dropoff_weekday delta_time	speed_mph
def get_time_interval(values):
    time_interval = 0
    if (values[4] > '07:00:00') & (values[4] <= '10:00:00'):
        time_interval = 1
    elif (values[4] > '10:00:00') & (values[4] <= '17:00:00'):
        time_interval = 2
    elif (values[4] > '17:00:00') & (values[4] <= '21:00:00'):
        time_interval = 3
    else: 
        time_interval = 4

    is_workday = 0
    if values[5] < 6:
        is_workday == 1
    else:
        is_workday == 0

    return Row(values[0],values[1],values[2],values[3], values[4], values[5],values[6], values[7],values[8],values[9],values[10],time_interval,is_workday)

weekday_df2 = weekday_df.rdd.map(get_time_interval)\
    .toDF(['PULocationID',	'DOLocationID',	'trip_distance',	\
        'pickup_date',	'pickup_time',	'pickup_weekday',	'dropoff_date',	\
        'dropoff_time',	'dropoff_weekday','delta_time','speed_mph','time_interval','is_workday'])



#### 首先按照weekday和time_interval进行研究
在周末，工作日的四个不同时段分别统计各个路段之间的以下指标：
- 总数：反应这个路段在该时间段内的数量
- 速度平均值：反应路段的拥堵程度


In [257]:
df1 = weekday_df2.groupBy('PULocationID',	'DOLocationID', 'is_workday','time_interval')\
    .agg(
        avg("speed_mph").alias("avg_speed_mph"), \
        count("speed_mph").alias("num")
    )


sum(num)
6896317


22/11/16 21:15:40 WARN TransportChannelHandler: Exception in connection from /10.12.7.110:54634
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:378)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:258)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:722)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:658)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.java:

In [252]:
df1.sort(col("num").desc())
# df1.groupBy().sum("num") 求和验证发现是对的

PULocationID,DOLocationID,is_workday,time_interval,avg_speed_mph,num
237,236,0,2,3.6554591755530796,25132
236,237,0,2,3.3820042867766062,22861
236,236,0,2,1.3649904345449577,21954
237,237,0,2,3.4325349774721365,21085
237,236,0,3,4.563869334809776,12643
264,264,0,2,10.080058698842329,12266
186,230,0,2,2.333851641818348,10933
239,142,0,2,2.388852883992223,10801
237,237,0,3,1.3371488033298646,10571
236,237,0,3,3.4322326742177003,10418


In [246]:
workday = weekday_df2[weekday_df2["pickup_weekday"] < 6]
weekend = weekday_df2[weekday_df2["pickup_weekday"] > 5]
workday.groupBy('time_interval').count()

weekend.groupBy('time_interval').count()
workday
lct_titvl_count = workday.groupBy("PULocationID","DOLocationID","time_interval").count().sort(col('count').desc())
zones.count()

#### 于zones中的数据进行连接

In [215]:
lct_zones = lct_titvl_count.alias("index").join(zones.alias("zones_start"),\
    col("index.PULocationID") == col("zones_start.LocationID"))\
    .join(zones.alias("zones_end"),\
    col("index.DOLocationID") == col("zones_end.LocationID"))\
    .select(col("index.PULocationID").alias("PULocationID"),\
    col("index.DOLocationID").alias("DOLocationID"),\
    col("time_interval"),\
    col("count"),\
    col("zones_start.Borough").alias("start.Borough"),\
    col("zones_start.Zone").alias("start.Zone"),\
    col("zones_start.service_zone").alias("start.service_zone"),\
    col("zones_end.Borough").alias("end.Borough"),\
    col("zones_end.Zone").alias("end.Zone"),\
    col("zones_end.service_zone").alias("end.service_zone")
    ).sort(col('count').desc())

# 后面的是终点
lct_zones

PULocationID,DOLocationID,time_interval,count,start.Borough,start.Zone,start.service_zone,end.Borough,end.Zone,end.service_zone
237,236,2,18881,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
236,237,2,17331,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,236,2,17093,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
237,237,2,16165,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
237,236,3,9957,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
264,264,2,8454,Unknown,NV,N/A,Unknown,NV,N/A
237,237,3,8345,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,237,3,8007,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,236,3,7915,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
237,161,2,7407,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Midtown Center,Yellow Zone


In [217]:
lct_zones

PULocationID,DOLocationID,time_interval,count,start.Borough,start.Zone,start.service_zone,end.Borough,end.Zone,end.service_zone
237,236,2,18881,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
236,237,2,17331,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,236,2,17093,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
237,237,2,16165,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
237,236,3,9957,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
264,264,2,8454,Unknown,NV,N/A,Unknown,NV,N/A
237,237,3,8345,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,237,3,8007,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side S...,Yellow Zone
236,236,3,7915,Manhattan,Upper East Side N...,Yellow Zone,Manhattan,Upper East Side N...,Yellow Zone
237,161,2,7407,Manhattan,Upper East Side S...,Yellow Zone,Manhattan,Midtown Center,Yellow Zone


### 存一些关键数据
- `lct_zones`
- `weekday_df2`
- `dis4_avg`

In [228]:
#lct_zones.toPandas().to_csv("./data/lct_zones.csv")
# weekday_df2.toPandas().to_csv("./data/weekday_df2.csv")
# dis4_avg.toPandas().to_csv("./data/dis4_avg.csv")

import pickle
# 保存
with open('./data/weekday_df2', 'wb') as f:
    pickle.dump(weekday_df2, f)

RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

## 可视化

# Bonus